# Assignment 1

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: POS tagging, Sequence labelling, RNNs

In [ ]:
#!pip install keras

In [1]:
# TODO remove
# from google.colab import drive
# drive.mount("/content/drive")

import pandas as pd
import numpy as np
import io
from pathlib import Path
import shutil
import urllib
import sys
import zipfile
import json

import tqdm
import random
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import load_model

# from keras.preprocessing.text import Tokenizer
from keras.layers import TextVectorization
from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing import sequence
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    confusion_matrix,
    classification_report,
)
import os
from typing import List, Callable, Dict

In [2]:
def set_reproducibility(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # os.environ["TF_DETERMINISTIC_OPS"] = "1" ### can make training slower


set_reproducibility(seed=42)

# [Task 1 - 0.5 points] Corpus

### Download the corpus

In [3]:
def download_url(download_path: Path, url: str):
    urllib.request.urlretrieve(url, filename=download_path)

In [4]:
dataset_url = "https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/packages/corpora/dependency_treebank.zip"
dataset_name = "dependency_treebank"

# print(f"Current work directory: {Path.cwd()}")
dataset_folder = Path.cwd().joinpath("Datasets")
if not dataset_folder.exists():
    dataset_folder.mkdir(parents=True)

dataset_zip_path = dataset_folder.joinpath("dependency_treebank.zip")
if not dataset_zip_path.exists():
    print("Downloading dataset... ", end="")
    download_url(url=dataset_url, download_path=dataset_zip_path)
    print("Download complete!")
else:
    print("Dataset already downloaded!")
dataset_path = dataset_folder.joinpath(dataset_name)

if not dataset_path.exists():
    print("Extracting dataset... (it may take a while...) ", end="")
    shutil.unpack_archive(dataset_zip_path, dataset_folder)
    print("Extraction completed!")
else:
    print("Dataset already extracted!")

Dataset already downloaded!
Dataset already extracted!


#### Encode the corpus into a pandas DataFrame object

In [5]:
folder = dataset_folder.joinpath(dataset_name)


dataframe_rows = []
for file_path in sorted(folder.glob("*.dp")):
    with file_path.open(mode="r", encoding="utf-8") as text_file:
        # Reading the text
        text = text_file.read()
        # Split sentences (\n\n is used for most NLP datasets to split sentences)
        sentences = text.split("\n\n")

        # Observing each sentence
        for s in sentences:
            sentence = []
            tags = []
            # sentence = [pierre,vinken,,aksjdajs, ]. tags = [NNP,aab,asd....]
            # Taking every line
            for line in s.split("\n"):
                columns = line.split("\t")
                # If every line have word, tag, value
                if len(columns) > 2:
                    # Put words and tags into lists
                    sentence.append(columns[0])
                    tags.append(columns[1])

            # Get the File_ID
            file_id = int(file_path.stem.split("_")[1])
            dataframe_row = {"file_id": file_id, "sentence": sentence, "tag": tags}
            dataframe_rows.append(dataframe_row)
# Create the dataframe
df = pd.DataFrame(dataframe_rows)

FILE_ID, SENTENCE, TAGS = df.columns.values

In [6]:
df.head()

,file_id,sentence,tag
0,1,"[Pierre, Vinken, ,, 61, years, old, ,, will, j...","[NNP, NNP, ,, CD, NNS, JJ, ,, MD, VB, DT, NN, ..."
1,1,"[Mr., Vinken, is, chairman, of, Elsevier, N.V....","[NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, NNP, ..."
2,2,"[Rudolph, Agnew, ,, 55, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, IN, NNP..."
3,3,"[A, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS..."
4,3,"[The, asbestos, fiber, ,, crocidolite, ,, is, ...","[DT, NN, NN, ,, NN, ,, VBZ, RB, JJ, IN, PRP, V..."


#### Splitting Data Train-Test-Validation

In [7]:
### file indices for train/validation/test
train_ids = np.arange(1, 101)
val_ids = np.arange(101, 151)
test_ids = np.arange(151, 200)

df_train = df[df[FILE_ID].isin(train_ids)]
df_val = df[df[FILE_ID].isin(val_ids)]
df_test = df[df[FILE_ID].isin(test_ids)]

# [Task 2 - 0.5 points] Text encoding

To train a neural POS tagger, you first need to encode text into numerical format.

### Instructions

* Embed words using **GloVe embeddings**.
* You are **free** to pick any embedding dimension.
* [Optional] You are free to experiment with text pre-processing: **make sure you do not delete any token!**

In [ ]:
import urllib
import zipfile

### download Glove

glove_file = "glove.6B.100d.txt"
glove_zip = "glove.6B.zip"
glove_path = Path.cwd().joinpath(glove_zip)
if not glove_path.exists():
    urllib.request.urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", glove_zip)
    with zipfile.ZipFile(glove_zip, "r") as zip_ref:
        zip_ref.extractall()

In [ ]:
### Download glove

embeddings_index = {}
with open(glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
# TODO secondo me è meglio così
# max_sequence_length = int(np.quantile([len(seq) for seq in df_train["sentence"]], 0.99))
max_sequence_length = np.max([len(seq) for seq in df_train[SENTENCE]])
### a little

# TODO remove
hparams = {
    "batch_size": 128,
    "embedding_dim": 100,
    "embedding_trainable": False,
    "learning_rate": 0.005,
    "max_sequence_length": max_sequence_length,
    "vocab_size": 7405,
    "tag_size": 46,
}

In [ ]:
### Preprocessing: lowercase TODO remove
def make_lower(df):
    df[SENTENCE] = df.map(lambda x: x.lower())


make_lower(df_train)
make_lower(df_val)
# TODO va bene fare lowercase sul test set? vediamo se va senza
# make_lower(df_test)

In [ ]:
### Use Keras Tokenizer to create Vocabulary

# ci hanno consigliato codice deprecato (tokenizer) io uso TextVectorization
# dobbiamo paddare

batch_size = 128


### tokenization
tokenizer = TextVectorization(
    max_tokens=20000,  # TODO
    output_sequence_length=max_sequence_length,
    standardize="lower_and_strip_punctuation",
    pad_to_max_tokens=True,  # TODO
)
ds_train = tf.data.Dataset.from_tensor_slices(df_train[SENTENCE].to_dict()).batch(
    batch_size
)
tokenizer.adapt(ds_train)

voc = tokenizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
tag_tokenizer = TextVectorization(
    max_tokens=100,
    output_sequence_length=np.max([len(seq) for seq in df_train[TAGS]]),
    standardize=None,
    pad_to_max_tokens=False,
)
ds_train = tf.data.Dataset.from_tensor_slices(df_train[TAGS].to_dict()).batch(
    batch_size
)
tag_tokenizer.adapt(ds_train)
tag_size = len(tag_tokenizer.get_vocabulary())

In [ ]:
embeddings_index = {}
with open(glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

In [ ]:
from keras.layers import Embedding

# embedding_layer = Embedding(
#    num_tokens,
#    embedding_dim,
#    trainable=False,
# )
# embedding_layer.build((1,))
# embedding_layer.set_weights([embedding_matrix])

In [ ]:
# TODO capirci qualcosa, fare un po' di inspection.

In [ ]:
### categorical (one hot) encoding of the labels
from keras.utils import to_categorical

num_classes = tag_size
y_train = to_categorical(tag_train, num_classes)
y_test = to_categorical(tag_test, num_classes)
y_val = to_categorical(tag_val, num_classes)

# [Task 3 - 1.0 points] Model definition

You are now tasked to define your neural POS tagger.

### Instructions

* **Baseline**: implement a Bidirectional LSTM with a Dense layer on top.
* You are **free** to experiment with hyper-parameters to define the baseline model.

* **Model 1**: add an additional LSTM layer to the Baseline model.
* **Model 2**: add an additional Dense layer to the Baseline model.

* **Do not mix Model 1 and Model 2**. Each model has its own instructions.

**Note**: if a document contains many tokens, you are **free** to split them into chunks or sentences to define your mini-batches.

#### Baseline

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, TimeDistributed

In [ ]:
class CreateModel(tf.keras.Model):
    def __init__(self, lstm_units=64, add_lstm=False, add_dense=False):
        super().__init__()

        # embedding_layer = Embedding(
        #    num_tokens,
        #    embedding_dim,
        #    trainable=False,
        # )
        # embedding_layer.build((1,))
        # embedding_layer.set_weights([embedding_matrix])
        self.add_lstm = add_lstm
        self.add_dense = add_dense

        self.embedding_layer = Embedding(
            num_tokens,
            embedding_dim,
            # input_length=max_sequence_length,
            weights=[embedding_matrix],
            trainable=False,
        )

        self.bi_lstm = Bidirectional(LSTM(lstm_units, return_sequences=True))
        self.bi_lstm2 = Bidirectional(LSTM(lstm_units, return_sequences=True))

        self.dense2 = TimeDistributed(Dense(tag_size, activation="relu"))
        self.dense = TimeDistributed(Dense(tag_size, activation="softmax"))

    def call(self, input, training=False):
        x = self.embedding_layer(input)
        ### Model 1
        if self.add_lstm:
            x = self.bi_lstm(x)

        ### Model 2
        if self.add_dense:
            x = self.dense2(x)

        output = self.dense(x)
        return output

# [Task 4 - 1.0 points] Metrics

Before training the models, you are tasked to define the evaluation metrics for comparison.

### Instructions

* Evaluate your models using macro F1-score, compute over **all** tokens.
* **Concatenate** all tokens in a data split to compute the F1-score. (**Hint**: accumulate FP, TP, FN, TN iteratively) 
* **Do not consider punctuation and symbol classes** $\rightarrow$ [What is punctuation?](https://en.wikipedia.org/wiki/English_punctuation)

**Note**: What about OOV tokens?
   * All the tokens in the **training** set that are not in GloVe are **not** considered as OOV
   * For the remaining tokens (i.e., OOV in the validation and test sets), you have to assign them a **static** embedding.
   * You are **free** to define the static embedding using any strategy (e.g., random, neighbourhood, etc...)

# [Task 5 - 1.0 points] Training and Evaluation

You are now tasked to train and evaluate the Baseline, Model 1, and Model 2.

### Instructions

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Compute metrics on the validation set.
* Pick **at least** three seeds for robust estimation.
* Pick the **best** performing model according to the observed validation set performance.

# [Task 6 - 1.0 points] Error Analysis

You are tasked to evaluate your best performing model.

### Instructions

* Compare the errors made on the validation and test sets.
* Aggregate model errors into categories (if possible) 
* Comment the about errors and propose possible solutions on how to address them.

# [Task 7 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...
* You can upload **model weights** in a cloud repository and report the link in the report.

# FAQ

Please check this frequently asked questions before contacting us

### Trainable Embeddings

You are **free** to define a trainable or non-trainable Embedding layer to load the GloVe embeddings.

### Model architecture

You **should not** change the architecture of a model (i.e., its layers).

However, you are **free** to play with their hyper-parameters.

### Neural Libraries

You are **free** to use any library of your choice to implement the networks (e.g., Keras, Tensorflow, PyTorch, JAX, etc...)

### Keras TimeDistributed Dense layer

If you are using Keras, we recommend wrapping the final Dense layer with `TimeDistributed`.

### Error Analysis

Some topics for discussion include:
   * Model performance on most/less frequent classes.
   * Precision/Recall curves.
   * Confusion matrices.
   * Specific misclassified samples.

### Punctuation

**Do not** remove punctuation from documents since it may be helpful to the model.

You should **ignore** it during metrics computation.

If you are curious, you can run additional experiments to verify the impact of removing punctuation.

# The End